In [1]:
import pandas as pd

In [2]:
prev_assoc_table = "../data/previous_associations/associations.csv"

In [3]:
df = pd.read_csv(prev_assoc_table)

In [4]:
df.loc[df.any_previous_association==True]

,Gene,Gene_symbol,Group,Giant,gwas,MGI_obesity,Marenne,Akbari,Locke,any_previous_association
3,ENSG00000006283,CACNA1G,white british_high_bmi_3,False,False,True,False,False,False,True
11,ENSG00000055957,ITIH1,white british_high_bmi_3,False,True,False,False,False,False,True
12,ENSG00000060718,COL11A1,white british_high_bmi_3,False,True,False,False,False,False,True
14,ENSG00000066735,KIF26A,white british_high_bmi_3,False,True,False,False,False,False,True
23,ENSG00000085999,RAD54L,white british_high_bmi_3,False,True,False,False,False,False,True
42,ENSG00000111344,RASAL1,white british_high_bmi_3,False,True,False,False,False,False,True
60,ENSG00000124299,PEPD,white british_high_bmi_3,False,True,False,False,False,False,True
68,ENSG00000131018,SYNE1,white british_high_bmi_3,False,True,False,False,False,False,True
74,ENSG00000134982,APC,white british_high_bmi_3,False,True,False,False,False,False,True
90,ENSG00000145362,ANK2,white british_high_bmi_3,False,True,True,False,False,False,True
